In [1]:
import os
import fire
import random
from retry.api import retry_call
from tqdm import tqdm
from datetime import datetime
from functools import wraps
from stylegan2_pytorch.stylegan2_pytorch_unet import Trainer, NanException

import torch
import torch.multiprocessing as mp
import torch.distributed as dist

import numpy as np

In [2]:
def cast_list(el):
    return el if isinstance(el, list) else [el]

def timestamped_filename(prefix = 'generated-'):
    now = datetime.now()
    timestamp = now.strftime("%m-%d-%Y_%H-%M-%S")
    return f'{prefix}{timestamp}'

def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

def run_training(rank, world_size, model_args, data, load_from, new, num_train_steps, name, seed):
    is_main = rank == 0
    is_ddp = world_size > 1

    if is_ddp:
        set_seed(seed)
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        dist.init_process_group('nccl', rank=rank, world_size=world_size)

        print(f"{rank + 1}/{world_size} process initialized.")

    model_args.update(
        is_ddp = is_ddp,
        rank = rank,
        world_size = world_size
    )

    model = Trainer(**model_args)

    if not new:
        model.load(load_from)
    else:
        model.clear()

    model.set_data_src(data)

    progress_bar = tqdm(initial = model.steps, total = num_train_steps, mininterval=10., desc=f'{name}<{data}>')
    while model.steps < num_train_steps:
        retry_call(model.train, tries=3, exceptions=NanException)
        progress_bar.n = model.steps
        progress_bar.refresh()
        if is_main and model.steps % 50 == 0:
            model.print_log()

    model.save(model.checkpoint_num)

    if is_ddp:
        dist.destroy_process_group()

def train_from_folder(
    data = '/home/mati/Desktop/workplace/datasets/CelebA-img',
    results_dir = './results',
    models_dir = './models',
    name = 'default',
    new = True,
    load_from = -1,
    image_size = 64,
    network_capacity = 16,
    fmap_max = 512,
    transparent = False,
    batch_size = 10,
    gradient_accumulate_every = 3,
    num_train_steps = 150000,
    learning_rate = 2e-4,
    lr_mlp = 0.1,
    ttur_mult = 1.5,
    rel_disc_loss = False,
    num_workers =  None,
    save_every = 1000,
    evaluate_every = 1000,
    generate = False,
    num_generate = 1,
    generate_interpolation = False,
    interpolation_num_steps = 100,
    save_frames = False,
    num_image_tiles = 8,
    trunc_psi = 0.75,
    mixed_prob = 0.9,
    fp16 = False,
    no_pl_reg = False,
    cl_reg = False,
    fq_layers = [],
    fq_dict_size = 256,
    attn_layers = [1,2],
    no_const = False,
    aug_prob = 0.,
    aug_types = ['translation', 'cutout'],
    top_k_training = False,
    generator_top_k_gamma = 0.99,
    generator_top_k_frac = 0.5,
    dual_contrast_loss = False,
    dataset_aug_prob = 0.,
    multi_gpus = False,
    calculate_fid_every = None,
    calculate_fid_num_images = 12800,
    clear_fid_cache = False,
    seed = 42,
    log = False
):
    model_args = dict(
        name = name,
        results_dir = results_dir,
        models_dir = models_dir,
        batch_size = batch_size,
        gradient_accumulate_every = gradient_accumulate_every,
        image_size = image_size,
        network_capacity = network_capacity,
        fmap_max = fmap_max,
        transparent = transparent,
        lr = learning_rate,
        lr_mlp = lr_mlp,
        ttur_mult = ttur_mult,
        rel_disc_loss = rel_disc_loss,
        num_workers = num_workers,
        save_every = save_every,
        evaluate_every = evaluate_every,
        num_image_tiles = num_image_tiles,
        trunc_psi = trunc_psi,
        fp16 = fp16,
        no_pl_reg = no_pl_reg,
        cl_reg = cl_reg,
        fq_layers = fq_layers,
        fq_dict_size = fq_dict_size,
        attn_layers = attn_layers,
        no_const = no_const,
        aug_prob = aug_prob,
        aug_types = cast_list(aug_types),
        top_k_training = top_k_training,
        generator_top_k_gamma = generator_top_k_gamma,
        generator_top_k_frac = generator_top_k_frac,
        dual_contrast_loss = dual_contrast_loss,
        dataset_aug_prob = dataset_aug_prob,
        calculate_fid_every = calculate_fid_every,
        calculate_fid_num_images = calculate_fid_num_images,
        clear_fid_cache = clear_fid_cache,
        mixed_prob = mixed_prob,
        log = log
    )

    world_size = torch.cuda.device_count()

    if world_size == 1 or not multi_gpus:
        run_training(0, 1, model_args, data, load_from, new, num_train_steps, name, seed)
        return

    mp.spawn(run_training,
        args=(world_size, model_args, data, load_from, new, num_train_steps, name, seed),
        nprocs=world_size,
        join=True)

In [ ]:
train_from_folder()

default</home/mati/Desktop/workplace/datasets/CelebA-img>:   0%|          | 0/150000 [00:00<?, ?it/s]/home/mati/anaconda3/envs/pytorch-gpu-2023-11/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392026823/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
default</home/mati/Desktop/workplace/datasets/CelebA-img>:   0%|          | 50/150000 [00:48<40:08:37,  1.04it/s]

G: 30.87 | D: 0.76 | GP: 0.21


default</home/mati/Desktop/workplace/datasets/CelebA-img>:   0%|          | 100/150000 [01:32<38:29:06,  1.08it/s]

G: 3.72 | D: 0.54 | GP: 2.14


default</home/mati/Desktop/workplace/datasets/CelebA-img>:   0%|          | 150/150000 [02:19<38:35:20,  1.08it/s]

...

default</home/mati/Desktop/workplace/datasets/CelebA-img>:  15%|█▌        | 22650/150000 [5:38:02<31:40:39,  1.12it/s]

G: -0.24 | D: 1.70 | GP: 0.14 | PL: 0.12


default</home/mati/Desktop/workplace/datasets/CelebA-img>:  15%|█▌        | 22700/150000 [5:38:56<31:40:46,  1.12it/s]

G: -0.08 | D: 1.72 | GP: 0.45 | PL: 0.12


default</home/mati/Desktop/workplace/datasets/CelebA-img>:  15%|█▌        | 22742/150000 [5:39:43<31:41:03,  1.12it/s]

In [6]:
import torch.nn as nn
from math import log2
from functools import partial

In [12]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, stride=2, padding=1)
        self.act = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.conv(x)
        x = self.act(x)
        return x

class Encoder(nn.Module):
    def __init__(self, image_size,in_channels=3, network_capacity=16, fmap_max=512):
        super().__init__()
        self.num_layers = int(log2(image_size) - 2) # Adjusting number of layers

        filters = [network_capacity * (2 ** i) for i in range(self.num_layers)]
        set_fmap_max = partial(min, fmap_max)
        filters = list(map(set_fmap_max, filters))
        self.blocks = nn.ModuleList([])
        for out_channels in filters:
            block = EncoderBlock(in_channels, out_channels)
            self.blocks.append(block)
            in_channels = out_channels

    def forward(self, x):
        skip_connections = []
        for block in self.blocks:
            x = block(x)
            skip_connections.append(x)
        return skip_connections

In [13]:
encoder = Encoder(image_size=64)

In [14]:
dummy_input = torch.randn(1, 3, 64, 64)

In [15]:
skip_connections = encoder(dummy_input)

skip_connections_shapes = [skip.shape for skip in skip_connections]

In [16]:
skip_connections_shapes

[torch.Size([1, 16, 32, 32]),
 torch.Size([1, 32, 16, 16]),
 torch.Size([1, 64, 8, 8]),
 torch.Size([1, 128, 4, 4])]